In [57]:
import pandas as pd
import numpy as np
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sample DataFrame for reference
np.random.seed(42)
df = pd.DataFrame({
    'Year': np.random.choice(['21/22', '22/23'], 100),
    'Program': np.random.choice(['Program1', 'Program2', 'Program3', 'Program4'], 100),
    'BSA': np.random.choice(['STGA', 'NE', 'PS', 'DI'], 100),
    'CourseA1': np.random.uniform(0, 10, 100),
    'CourseB1': np.random.uniform(0, 10, 100),
    'CourseC1': np.random.uniform(0, 10, 100),
    'CourseD1': np.random.uniform(0, 10, 100),
    'CourseA2': np.random.uniform(0, 10, 100),
    'CourseB2': np.random.uniform(0, 10, 100),
    'CourseC2': np.random.uniform(0, 10, 100),
    'CourseD2': np.random.uniform(0, 10, 100),
    'CourseA3': np.random.uniform(0, 10, 100),
    'CourseB3': np.random.uniform(0, 10, 100),
    'CourseC3': np.random.uniform(0, 10, 100),
    'CourseD3': np.random.uniform(0, 10, 100),
    'CourseA4': np.random.uniform(0, 10, 100),
    'CourseB4': np.random.uniform(0, 10, 100),
    'CourseC4': np.random.uniform(0, 10, 100),
    'CourseD4': np.random.uniform(0, 10, 100)
})

# Define the passing grade
passing_grade = 5.5

# Define the courses for each program
courses = {
    'Program1': ['CourseA1', 'CourseB1', 'CourseC1', 'CourseD1'],
    'Program2': ['CourseA2', 'CourseB2', 'CourseC2', 'CourseD2'],
    'Program3': ['CourseA3', 'CourseB3', 'CourseC3', 'CourseD3'],
    'Program4': ['CourseA4', 'CourseB4', 'CourseC4', 'CourseD4']
}

# Create pass/fail indicators and scenarios
df['Scenario'] = ''
for program, course_list in courses.items():
    for course in course_list:
        df[f'{course}_pass'] = (df[course] >= passing_grade).astype(int)
    df.loc[df['Program'] == program, 'Scenario'] = df[df['Program'] == program][[f'{course}_pass' for course in course_list]].astype(str).agg(''.join, axis=1)

# Define custom order of scenarios
scenarios = [1,0]

# Define the order of 'Program' facet columns and BSA categories
program_order = ['Program1', 'Program2', 'Program3', 'Program4']
bsa_order = ['STGA', 'NE', 'PS', 'DI']
bsa_colors = {
    'STGA': '#4d4d4d',
    'NE': '#b30000',
    'PS': '#258e25',
    'DI': 'lightblue'
}



# Create a figure with subplots arranged in a 8x1 grid
fig = make_subplots(rows=8, cols=4, subplot_titles=[f'{program} {year} {course}' for program in program_order for year in df['Year'].unique() for course in courses[program]], vertical_spacing=0.1)

# Loop through each 'Program' facet column and 'Year' facet row
row_counter = 1
for program in program_order:
    for year in df['Year'].unique():
        # Filter data for the current program and year
        df_subset = df[(df['Program'] == program) & (df['Year'] == year)]
        
        for i, course in enumerate(courses[program], start=1):
                # Prepare data for plotting
            df_subset2 = df_subset.groupby(['BSA', f'{course}_pass']).size().reset_index(name='Count')
            # Loop through each 'BSA' category
            for bsa in bsa_order:
                # Filter data for the current BSA category
                df_bsa = df_subset2[df_subset2['BSA'] == bsa]
                
                # Create bar trace
                bar = go.Bar(
                    x=pd.Categorical(df_bsa[f'{course}_pass'], categories=scenarios, ordered=True),
                    y=df_bsa['Count'],
                    name=bsa,
                    marker=dict(color=bsa_colors[bsa]),
                    text=df_bsa['Count'],  # Add text on top of each bar
                    textposition='auto'  # Automatically position the text
                )
                
                # Add trace to the subplot
                fig.add_trace(bar, row=row_counter, col=i)
        
        row_counter += 1

# Update layout
fig.update_layout(title='Count of Scenarios Layered by BSA and Program', height=2400, width=800)
fig.update_layout(bargap=0.2)
#fig.update_xaxes(title_text='Scenario')
fig.update_yaxes(title_text='Count')
fig.update_layout(showlegend=False)  # Hide the legend
fig.update_annotations(font_size=12)  # Update font size for subplot titles
fig.show()


